# Demo notebook for transmission analysis

Calculate which branches or transmission lines are congested by counting the fraction of total hours that the power flow is 0.75, 0.8 or 0.9 of the branch capacity. Z-score and p-values are calculated for the 0.75 case, since 0.75 is also the value used by WECC (useful for validation).


In [1]:
import pandas as pd
import numpy as np

from postreise.analyze.transmission import generate_cong_stats
from powersimdata.scenario.scenario import Scenario

Consider the base case scenario for this analysis.

In [2]:
scenario = Scenario('original')

SCENARIO: base | original

--> State
analyze
--> Loading Western interconnect
Loading zone
Loading sub
Loading bus2sub
Loading bus
Loading plant
Loading plant cost
Loading branch
Loading DC line


Load power flow data

In [3]:
pf = scenario.state.get_pf()
pf.head()

--> Loading PF


,88209,88210,88211,88212,88213,88214,88215,88216,88217,88218,...,100905,100906,100907,100908,100909,100910,100911,100912,100913,100914
UTC,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,13.410977,-5.514450,26.500863,1.093795,10.262175,20.955831,8.560032,-9.653827,-11.009266,24.340703,...,-1.777565e-08,-5.225758e-08,-104.67,-95.0,-95.0,-95.0,59.509675,-59.509675,24.910570,0.0
2016-01-01 01:00:00,14.553511,-6.021594,28.696669,1.186219,11.107621,22.657324,9.248811,-10.435031,-11.895122,29.144931,...,-4.993055e-08,-1.046700e+02,-104.67,-95.0,-95.0,-95.0,71.212200,-71.212200,30.270490,0.0
2016-01-01 02:00:00,15.941922,-6.877471,30.967848,1.330101,12.112802,24.313556,9.826090,-11.156192,-12.637575,29.696942,...,-1.888040e-07,-1.046700e+02,-104.67,-95.0,-95.0,-95.0,75.402644,-75.402644,32.116559,0.0
2016-01-01 03:00:00,15.754338,-6.787216,30.618925,1.314549,11.977067,24.048088,9.720584,-11.035132,-12.501880,29.462868,...,-6.870593e-08,-1.046700e+02,-104.67,-95.0,-95.0,-95.0,71.671863,-71.671863,30.428562,0.0
2016-01-01 04:00:00,15.314555,-6.593918,29.770550,1.278930,11.650145,23.388848,9.453388,-10.732318,-12.158234,29.462335,...,-6.825829e-08,-1.046700e+02,-104.67,-95.0,-95.0,-95.0,68.787608,-68.787608,29.170215,0.0


Retrieve the branch data frame that encloses information on the lines in the network.

In [4]:
branch = scenario.state.get_grid().branch.copy()

Create a capacity column in the data frame. Note that rateA == 0 represents low impedance lines.
Denoting rateA == 0 is simply a signal to MATPOWER that there are no current constraints on the line.

In [5]:
branch.loc[branch.rateA != 0, 'capacity'] = branch['rateA']
branch.loc[branch.rateA == 0, 'capacity'] = 99999.
branch.head()

,from_bus_id,to_bus_id,r,x,b,rateA,rateB,rateC,ratio,angle,...,interconnect,from_lat,from_lon,to_lat,to_lon,from_zone_id,to_zone_id,from_zone_name,to_zone_name,capacity
branch_id,,,,,,,,,,,,,,,,,,,,,
88209,2010002,2010001,0.057474,0.297874,0.04557,185.33,0,0,0.0,0.0,...,Western,47.695555,-124.183645,48.241400,-124.577777,201,201,Washington,Washington,185.33
88210,2010011,2010001,0.027895,0.200986,0.06271,167.88,0,0,0.0,0.0,...,Western,48.002536,-123.762001,48.241400,-124.577777,201,201,Washington,Washington,167.88
88211,2010014,2010002,0.028286,0.132954,0.07205,207.43,0,0,0.0,0.0,...,Western,47.188929,-123.686007,47.695555,-124.183645,201,201,Washington,Washington,207.43
88212,2010004,2010003,0.012099,0.073288,0.01171,170.67,0,0,0.0,0.0,...,Western,46.927528,-124.171950,47.040041,-124.056969,201,201,Washington,Washington,170.67
88213,2010003,2010010,0.012423,0.079127,0.01766,201.54,0,0,0.0,0.0,...,Western,47.040041,-124.056969,47.174349,-123.847397,201,201,Washington,Washington,201.54


Normalize power flow values to line capacity.

In [6]:
pf_norm = pf.div(branch['capacity']).apply(np.abs)
pf_norm.head()

,88209,88210,88211,88212,88213,88214,88215,88216,88217,88218,...,100905,100906,100907,100908,100909,100910,100911,100912,100913,100914
UTC,,,,,,,,,,,,,,,,,,,,,
2016-01-01 00:00:00,0.072363,0.032848,0.127758,0.006409,0.050919,0.098942,0.000086,0.049481,0.000110,0.114631,...,2.412875e-11,7.142623e-11,0.144156,0.142851,0.142892,0.143746,0.223385,0.274998,0.134870,0.0
2016-01-01 01:00:00,0.078528,0.035868,0.138344,0.006950,0.055114,0.106975,0.000092,0.053486,0.000119,0.137256,...,6.777596e-11,1.430641e-01,0.144156,0.142851,0.142892,0.143746,0.267313,0.329077,0.163890,0.0
2016-01-01 02:00:00,0.086019,0.040967,0.149293,0.007793,0.060101,0.114795,0.000098,0.057182,0.000126,0.139856,...,2.562834e-10,1.430641e-01,0.144156,0.142851,0.142892,0.143746,0.283043,0.348441,0.173885,0.0
2016-01-01 03:00:00,0.085007,0.040429,0.147611,0.007702,0.059428,0.113541,0.000097,0.056561,0.000125,0.138753,...,9.326175e-11,1.430641e-01,0.144156,0.142851,0.142892,0.143746,0.269039,0.331201,0.164746,0.0
2016-01-01 04:00:00,0.082634,0.039278,0.143521,0.007494,0.057806,0.110429,0.000095,0.055009,0.000122,0.138751,...,9.265412e-11,1.430641e-01,0.144156,0.142851,0.142892,0.143746,0.258212,0.317872,0.157933,0.0


Create congestion statistics file

In [7]:
congestion = generate_cong_stats(pf_norm,
                                 branch,
                                 '../data/congestion_stats_base_demo')

In [8]:
congestion.loc[congestion['pvalue'] < 0.05].sort_values(['pvalue', 'zscore'], ascending=False).head(10)

,capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil0p75-0p8,hutil>=0p9,hutil>=0p8,hutil>=0p75,dist,zscore,pvalue
100435,392,0,0,6,34,0,6,40,0.000000,2.677886,3.704420e-03
99647,169,0,7,18,17,7,25,42,0.000000,3.068534,1.075559e-03
99877,279,0,0,2,40,0,2,42,53.208235,3.068534,1.075559e-03
98544,476,0,0,1,45,0,1,46,48.364319,3.849830,5.910002e-05
98086,214,0,0,27,22,0,27,49,44.630752,4.435801,4.586522e-06
99315,567,0,0,6,43,0,6,49,111.365148,4.435801,4.586522e-06
91770,222,0,3,18,29,3,21,50,1.709758,4.631125,1.818419e-06
93812,268,0,0,4,47,0,4,51,43.555793,4.826449,6.949444e-07
99316,497,0,0,6,52,0,6,58,19.233258,6.193716,2.938095e-10
94549,231,0,0,6,55,0,6,61,3.461302,6.779688,6.021739e-12
